In [17]:
%load_ext kedro

The kedro extension is already loaded. To reload it, use:
  %reload_ext kedro


In [18]:
df = catalog.load('raw_data')
df.info()

[10/16/25 20:16:02] INFO     Loading data from raw_data (ParquetDataset)...                    ]8;id=685040;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=436863;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/io/data_catalog.py#1046\1046]8;;\

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55043 entries, 0 to 55042
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        55043 non-null  object
 1   category    55043 non-null  object
 2   item        55043 non-null  object
 3   unit        55043 non-null  object
 4   low_price   55043 non-null  object
 5   high_price  55043 non-null  object
dtypes: object(6)
memory usage: 2.5+ MB


In [19]:
import pandas as pd

**There seems to be duplicated data from 7 days.**

In [118]:
df.drop_duplicates(subset=['date', 'unit', 'item'], inplace=True)

In [119]:
df

,date,category,item,unit,low_price,high_price
0,2025-08-01,চাল,চাল সরু (নাজির/মিনিকেট),প্রতি কেজি,75,85
1,2025-08-01,চাল,চাল (মাঝারী)পাইজাম/আটাশ,প্রতি কেজি,60,75
2,2025-08-01,চাল,চাল (মোটা)/স্বর্ণা/চায়না ইরি,প্রতি কেজি,55,60
3,2025-08-01,আটা/ময়দা,আটা সাদা (খোলা),প্রতি কেজি,40,45
4,2025-08-01,আটা/ময়দা,আটা (প্যাকেট),প্রতি কেজি প্যাঃ,50,55
...,...,...,...,...,...,...
55038,2024-07-08,বিবিধঃ,লবণ(প্যাঃ)আয়োডিনযুক্ত(মানভেদে),প্রতি কেজি,38,42
55039,2024-07-08,বিবিধঃ,ডিম (ফার্ম),প্রতি হালি,45,50
55040,2024-07-08,বিবিধঃ,লেখার কাগজ(সাদা),প্রতি দিস্তা,30,35
55041,2024-07-08,বিবিধঃ,"এম,এস রড (৬০ গ্রেড)",প্রতি মেঃটন,88500,89500


In [120]:
df['date'] = pd.to_datetime(df['date'])

In [121]:

numeric_cols = ['low_price', 'high_price']

for col in numeric_cols:
    print(f"\n=== CHECKING {col} ===")
    
    numeric_series = pd.to_numeric(df[col], errors='coerce')
    non_numeric_mask = numeric_series.isna() & df[col].notna()
    
    if non_numeric_mask.any():
        print(f"Non-numeric values found in {col}:")
        non_numeric_values = df.loc[non_numeric_mask, col].unique()
        print(non_numeric_values)
        print(f"Count: {non_numeric_mask.sum()}")
    else:
        print(f"All values in {col} are numeric!")


=== CHECKING low_price ===
Non-numeric values found in low_price:
['বাজারে পাওয়া']
Count: 1

=== CHECKING high_price ===
Non-numeric values found in high_price:
['যায়নি']
Count: 1


In [122]:
df.loc[df['low_price'] == 'বাজারে পাওয়া', 'low_price']


42669    বাজারে পাওয়া
Name: low_price, dtype: object

In [123]:
df.loc[df['high_price'] == 'যায়নি', 'high_price']


42669    যায়নি
Name: high_price, dtype: object

**there seems to be one row with non numeric column. possible typo**

In [124]:
df['low_price'] = pd.to_numeric(df['low_price'], errors='coerce')
df['high_price'] = pd.to_numeric(df['high_price'], errors='coerce')

In [125]:
df['unit'].value_counts()


unit
প্রতি কেজি          38071
১ কেজি               4116
প্রতি লিটার          3085
প্রতি কেজি প্যাঃ     2058
প্রতি মেঃটন          2058
৫ লিটার              1490
১ লিটার              1357
প্রতি দিস্তা         1029
প্রতি হালি           1029
2 লিটার               370
50.00-52.00             2
Name: count, dtype: int64

In [126]:
df[df['unit'] == '50.00-52.00']

,date,category,item,unit,low_price,high_price
5664,2023-11-13,মসলাঃ,তেজপাতা,50.00-52.00,150.0,200.0
41850,2023-11-14,মসলাঃ,তেজপাতা,50.00-52.00,150.0,200.0


**some units seem to have been mislabeled. ahve to fill them with proper unit for the item.**

In [127]:
df[df['item'] == 'তেজপাতা']['unit']


36       প্রতি কেজি
91       প্রতি কেজি
146      প্রতি কেজি
198      প্রতি কেজি
255      প্রতি কেজি
            ...    
54811    প্রতি কেজি
54865    প্রতি কেজি
54917    প্রতি কেজি
54971    প্রতি কেজি
55025    প্রতি কেজি
Name: unit, Length: 1029, dtype: object

In [128]:
df[df['unit'] == '50.00-52.00']['unit'] = 'প্রতি কেজি'

                    WARNING  /tmp/ipykernel_6571/905726389.py:1: SettingWithCopyWarning:            ]8;id=61830;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=687678;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/warnings.py#110\110]8;;\
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc = value instead                                                        
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               df[df['unit'] == '50.00-52.00']['unit'] = 'প্রতি কেজি'                                  
                                                                                                                   

In [129]:
unit_mapping = {
    'প্রতি কেজি': '১ কেজি',
    'প্রতি লিটার': '১ লিটার',
    'প্রতি কেজি প্যাঃ': '১ কেজি',
    'প্রতি মেঃটন': '১ মেঃটন',
    'প্রতি দিস্তা': '১ দিস্তা',
    'প্রতি হালি': '১ হালি',
}

df['unit'] = df['unit'].replace(unit_mapping)

In [130]:
df['unit'].value_counts()


unit
১ কেজি         44245
১ লিটার         4442
১ মেঃটন         2058
৫ লিটার         1490
১ দিস্তা        1029
১ হালি          1029
2 লিটার          370
50.00-52.00        2
Name: count, dtype: int64

In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54665 entries, 0 to 55042
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        54665 non-null  datetime64[ns]
 1   category    54665 non-null  object        
 2   item        54665 non-null  object        
 3   unit        54665 non-null  object        
 4   low_price   54664 non-null  float64       
 5   high_price  54664 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 2.9+ MB


In [132]:
df[df['unit'].isna()]

,date,category,item,unit,low_price,high_price


In [133]:
df['category'].value_counts()


category
মসলাঃ                   16464
ডাল                      7203
বিবিধঃ                   7203
ভোজ্য তেল                6302
মাছ ও গোশত:              6174
আটা/ময়দা                 4116
গুড়া দুধ(প্যাকেটজাত)     4116
চাল                      3087
Name: count, dtype: int64

In [134]:
category_mapping = {
    'মসলাঃ': 'মসলা',
    'বিবিধঃ': 'বিবিধ', 
    'মাছ ও গোশত:': 'মাছ/গোশত',
}

df['category'] = df['category'].replace(category_mapping)

In [135]:
df['category'].value_counts()


category
মসলা                    16464
ডাল                      7203
বিবিধ                    7203
ভোজ্য তেল                6302
মাছ/গোশত                 6174
আটা/ময়দা                 4116
গুড়া দুধ(প্যাকেটজাত)     4116
চাল                      3087
Name: count, dtype: int64

In [136]:
df['item'].value_counts()


item
সয়াবিন তেল (বোতল)               2392
চাল (মোটা)/স্বর্ণা/চায়না ইরি    1029
আটা সাদা (খোলা)                 1029
আটা (প্যাকেট)                   1029
চাল সরু (নাজির/মিনিকেট)         1029
                                ... 
রসুন(দেশী)                        60
আলু (নতুন/পুরাতন)(মানভেদে)        41
রসুন(দেশী) পুরাতন                 29
রসুন(দেশী) নতুন/পুরাতন            29
পিঁয়াজ (নতুন/পুরাতন) (দেশী)        7
Name: count, Length: 69, dtype: int64

In [137]:
for item in df['item'].unique():
    print(item)

চাল সরু (নাজির/মিনিকেট)
চাল (মাঝারী)পাইজাম/আটাশ
চাল (মোটা)/স্বর্ণা/চায়না ইরি
আটা সাদা (খোলা)
আটা (প্যাকেট)
ময়দা (খোলা)
ময়দা (প্যাকেট)
সয়াবিন তেল (লুজ)
সয়াবিন তেল (বোতল)
পাম অয়েল (লুজ)
সুপার পাম অয়েল (লুজ)
রাইস ব্রান তেল (বোতল)
মশুর ডাল (বড় দানা)
মশুর ডাল (মাঝারী দানা)
মশুর ডাল (ছোট দানা)
মুগ ডাল (মানভেদে)
এ্যাংকর ডাল
ছোলা (মানভেদে)
আলু (মানভেদে)
পিঁয়াজ (দেশী)
পিঁয়াজ (আমদানি)
রসুন (দেশী)
রসুন (আমদানি)
শুকনা মরিচ (দেশী)
শুকনা মরিচ (আমদানি)
হলুদ (দেশী)
হলুদ (আমদানি)
আদা (দেশী)
আদা (আমদানি)
জিরা
দারুচিনি
লবঙ্গ
এলাচ(ছোট)
ধনে
তেজপাতা
রুই
ইলিশ
গরু
খাসী
মুরগী(ব্রয়লার)
মুরগী (দেশী)
ডানো
ডিপ্লোমা (নিউজিল্যান্ড)
ফ্রেশ
মার্কস
চিনি
খেজুর(সাধারণ মানের)
লবণ(প্যাঃ)আয়োডিনযুক্ত
ডিম (ফার্ম)
লেখার কাগজ(সাদা)
এম,এস রড (৬০ গ্রেড)
এম,এস রড( ৪০ গ্রেড)
পাম অয়েল (সুপার)
মশূর ডাল (মাঝারী দানা)
লবণ(প্যাঃ)আয়োডিনযুক্ত(মানভেদে)
চাল (মাঝারী)পাইজাম/লতা
রসুন(দেশী) নতুন/পুরাতন)
পাম অলিন (বোতল)
পিঁয়াজ (নতুন) (দেশী)
রসুন(দেশী)
আলু (নতুন, মানভেদে)
রসুন(দেশী) নতুন
আদা (দেশী)(নতুন)
আদা (দেশী) নতুন
আলু (নতুন/পুরাতন)(মানভেদে)

**There seems to be some redundent names that need merging**

In [138]:
pd.set_option('display.max_rows', 69)
item_counts = df['item'].value_counts()
print(item_counts)
pd.reset_option('display.max_rows')

item
সয়াবিন তেল (বোতল)                 2392
চাল (মোটা)/স্বর্ণা/চায়না ইরি      1029
আটা সাদা (খোলা)                   1029
আটা (প্যাকেট)                     1029
চাল সরু (নাজির/মিনিকেট)           1029
চিনি                              1029
ময়দা (খোলা)                       1029
ময়দা (প্যাকেট)                    1029
পাম অয়েল (লুজ)                    1029
মশুর ডাল (বড় দানা)                1029
এ্যাংকর ডাল                       1029
মুগ ডাল (মানভেদে)                 1029
মশুর ডাল (ছোট দানা)               1029
পিঁয়াজ (আমদানি)                   1029
ছোলা (মানভেদে)                    1029
হলুদ (আমদানি)                     1029
আদা (আমদানি)                      1029
হলুদ (দেশী)                       1029
শুকনা মরিচ (আমদানি)               1029
শুকনা মরিচ (দেশী)                 1029
রসুন (আমদানি)                     1029
ধনে                               1029
তেজপাতা                           1029
লবঙ্গ                             1029
এলাচ(ছোট)                        1029
রুই                 

In [139]:
def analyze_merge_possibilities_exact_dates(df):
    
    item_groups = {
        'rice': ['চাল (মাঝারী)পাইজাম/লতা', 'চাল (মাঝারী)পাইজাম/আটাশ'],
        'garlic_local': ['রসুন (দেশী)', 'রসুন(দেশী)', 'রসুন(দেশী) নতুন/পুরাতন)', 'রসুন(দেশী) নতুন', 'রসুন (দেশী) নতুন/পুরাতন', 'রসুন(দেশী) নতুন/পুরাতন', 'রসুন(দেশী) পুরাতন'],
        'ginger_local': ['আদা (দেশী)', 'আদা (দেশী)(নতুন)', 'আদা (দেশী) নতুন'],
        'onion_local': ['পিঁয়াজ (দেশী)', 'পিঁয়াজ (নতুন) (দেশী)', 'পিঁয়াজ (নতুন/পুরাতন) (দেশী)'],
        'potato': ['আলু (মানভেদে)', 'আলু (নতুন, মানভেদে)', 'আলু (নতুন/পুরাতন)(মানভেদে)'],
        'lentils_medium': ['মশুর ডাল (মাঝারী দানা)', 'মশূর ডাল (মাঝারী দানা)'],
        'salt': ['লবণ(প্যাঃ)আয়োডিনযুক্ত', 'লবণ(প্যাঃ)আয়োডিনযুক্ত(মানভেদে)'],
        'palm_oil_super': ['পাম অয়েল (সুপার)', 'সুপার পাম অয়েল (লুজ)'],
        'soybean_oil_bottle': ['সয়াবিন তেল (বোতল)'],  # Could merge different sizes if they exist
        'chicken_broiler': ['মুরগী(ব্রয়লার)'],  # Standardize naming
        'chicken_local': ['মুরগী (দেশী)'],  # Standardize naming
        'ms_rod': ['এম,এস রড (৬০ গ্রেড)', 'এম,এস রড( ৪০ গ্রেড)']  # Different grades but same product type
    }
    
    merge_suggestions = {}
    
    for group_name, items in item_groups.items():
        print(f"\n{'='*60}")
        print(f"ANALYZING MERGE POSSIBILITIES: {group_name.upper()}")
        print(f"{'='*60}")
        
        date_sets = {}
        for item in items:
            if item in df['item'].values:
                unique_dates = set(df[df['item'] == item]['date'].dt.date)
                date_sets[item] = unique_dates
                print(f"{item}: {len(unique_dates)} unique dates from {min(unique_dates)} to {max(unique_dates)}")
        
        items_list = list(date_sets.keys())
        possible_merges = []
        
        for i in range(len(items_list)):
            for j in range(i + 1, len(items_list)):
                item1, item2 = items_list[i], items_list[j]
                set1 = date_sets[item1]
                set2 = date_sets[item2]
                
                are_disjoint = set1.isdisjoint(set2)
                
                if are_disjoint:
                    possible_merges.append((item1, item2))
                    print(f"✅ CAN MERGE: {item1} + {item2} (completely disjoint dates)")
                else:
                    overlap_dates = set1.intersection(set2)
        
        merge_suggestions[group_name] = possible_merges
    
    return merge_suggestions

merge_analysis = analyze_merge_possibilities_exact_dates(df)


ANALYZING MERGE POSSIBILITIES: RICE
চাল (মাঝারী)পাইজাম/লতা: 749 unique dates from 2021-05-17 to 2024-10-29
চাল (মাঝারী)পাইজাম/আটাশ: 280 unique dates from 2024-10-30 to 2025-10-13
✅ CAN MERGE: চাল (মাঝারী)পাইজাম/লতা + চাল (মাঝারী)পাইজাম/আটাশ (completely disjoint dates)

ANALYZING MERGE POSSIBILITIES: GARLIC_LOCAL
রসুন (দেশী): 438 unique dates from 2021-09-01 to 2025-10-13
রসুন(দেশী): 60 unique dates from 2024-06-24 to 2024-08-31
রসুন(দেশী) নতুন/পুরাতন): 302 unique dates from 2022-02-18 to 2024-01-01
রসুন(দেশী) নতুন: 94 unique dates from 2024-03-04 to 2024-06-23
রসুন (দেশী) নতুন/পুরাতন: 77 unique dates from 2021-05-17 to 2021-08-27
রসুন(দেশী) নতুন/পুরাতন: 29 unique dates from 2024-02-01 to 2024-03-03
রসুন(দেশী) পুরাতন: 29 unique dates from 2024-01-02 to 2024-01-31
✅ CAN MERGE: রসুন (দেশী) + রসুন(দেশী) (completely disjoint dates)
✅ CAN MERGE: রসুন (দেশী) + রসুন(দেশী) নতুন/পুরাতন) (completely disjoint dates)
✅ CAN MERGE: রসুন (দেশী) + রসুন(দেশী) নতুন (completely disjoint dates)
✅ CAN MERG

In [ ]:
def create_safe_merge_map():
    merge_rules = {
        'চাল (মাঝারী)পাইজাম/আটাশ': 'চাল (মাঝারী)পাইজাম/লতা',
        
        'রসুন(দেশী)': 'রসুন (দেশী)',
        'রসুন(দেশী) নতুন/পুরাতন)': 'রসুন (দেশী)',
        'রসুন(দেশী) নতুন': 'রসুন (দেশী)',
        'রসুন (দেশী) নতুন/পুরাতন': 'রসুন (দেশী)',
        'রসুন(দেশী) নতুন/পুরাতন': 'রসুন (দেশী)',
        'রসুন(দেশী) পুরাতন': 'রসুন (দেশী)',
        
        'আদা (দেশী)(নতুন)': 'আদা (দেশী)',
        'আদা (দেশী) নতুন': 'আদা (দেশী)',
        
        'পিঁয়াজ (নতুন) (দেশী)': 'পিঁয়াজ (দেশী)',
        'পিঁয়াজ (নতুন/পুরাতন) (দেশী)': 'পিঁয়াজ (দেশী)',
        
        'আলু (নতুন, মানভেদে)': 'আলু (মানভেদে)',
        'আলু (নতুন/পুরাতন)(মানভেদে)': 'আলু (মানভেদে)',
        
        'মশুর ডাল (মাঝারী দানা)': 'মশূর ডাল (মাঝারী দানা)',
        
        'লবণ(প্যাঃ)আয়োডিনযুক্ত': 'লবণ(প্যাঃ)আয়োডিনযুক্ত(মানভেদে)',
        
        'সুপার পাম অয়েল (লুজ)': 'পাম অয়েল (সুপার)',
        
    }
    
    return merge_rules

def apply_safe_merging(df):
    merge_rules = create_safe_merge_map()
    
    print("Applying safe merging based on disjoint date analysis...")
    print(f"Total merge rules: {len(merge_rules)}")
    
    df['item'] = df['item'].replace(merge_rules)
    
    original_item_count = len(set(merge_rules.keys()))
    new_item_count = len(df['item'].unique())
    
    print(f"Reduced from {original_item_count} to {new_item_count} unique items")
    print(f"Items merged successfully!")
    
    return df

df_merged = apply_safe_merging(df)

Applying safe merging based on disjoint date analysis...
Total merge rules: 16
Reduced from 16 to 53 unique items
Items merged successfully!


In [141]:
cnt = df['item'].value_counts()
cnt


item
সয়াবিন তেল (বোতল)                 2392
চাল (মাঝারী)পাইজাম/লতা            1029
চাল (মোটা)/স্বর্ণা/চায়না ইরি      1029
আটা সাদা (খোলা)                   1029
চাল সরু (নাজির/মিনিকেট)           1029
আটা (প্যাকেট)                     1029
ময়দা (খোলা)                       1029
ময়দা (প্যাকেট)                    1029
পাম অয়েল (লুজ)                    1029
মশুর ডাল (বড় দানা)                1029
পাম অয়েল (সুপার)                  1029
মশুর ডাল (ছোট দানা)               1029
মশূর ডাল (মাঝারী দানা)            1029
ছোলা (মানভেদে)                    1029
আলু (মানভেদে)                     1029
মুগ ডাল (মানভেদে)                 1029
এ্যাংকর ডাল                       1029
পিঁয়াজ (দেশী)                     1029
পিঁয়াজ (আমদানি)                   1029
রসুন (আমদানি)                     1029
রসুন (দেশী)                       1029
গরু                               1029
খাসী                              1029
শুকনা মরিচ (দেশী)                 1029
শুকনা মরিচ (আমদানি)               1029
হলুদ (দেশী)        

In [142]:
def remove_items_with_few_days(df, min_days=1000):
    df_temp = df.copy()
    df_temp['date'] = pd.to_datetime(df_temp['date']).dt.date
    
    item_day_counts = df_temp.groupby('item')['date'].nunique()
    
    items_to_keep = item_day_counts[item_day_counts >= min_days].index
    df_filtered = df[df['item'].isin(items_to_keep)].copy()
    
    removed_count = len(df) - len(df_filtered)
    removed_items = set(df['item'].unique()) - set(items_to_keep)
    
    print(f"\n=== FILTERING RESULTS ===")
    print(f"Removed {removed_count} records from {len(removed_items)} items")
    print(f"Remaining items: {len(items_to_keep)}")
    print(f"Remaining records: {len(df_filtered)}")
    
    return df_filtered

df_cleaned = remove_items_with_few_days(df, min_days=1000)


=== FILTERING RESULTS ===
Removed 825 records from 2 items
Remaining items: 51
Remaining records: 53840
